# Predicting the winner of Races with XGBoost

## 1. Introduction
XGBoost is an open-source software library for predictive modelling, created by Tianqi Chen in 2014. The name XGBoost stands for "Extreme Gradient Boosting" and implements ”Gradient Boosting” as proposed in Greedy Function Approximation: A Gradient Boosting Machine by Friedman, while the term ”Extreme” refers to the engineering goal to maximize the resources used by the algorithm to achieve high accuracy, computational efficiency and scalability. What started off as a terminal application for a research project, has become a scalable end-to-end tree boosting system that has integrations with scikit-learn in Python, the caret package in R, as well as big data frameworks like Apache Spark and Hadoop. Since its introduction, XGBoost has been used in more than half of the winning solutions in Kaggle competitions, after it gained much popularity and attention in the community by winning the Higgs boson machine learning challenge.

Tree based algorithms are widely used in supervised machine learning. XGBoost is an implementation of gradient boosting, representing a new generation of GBM algorithms with tweaks to the traditional tree boosting and major system improvements. It has developed to the go-to algorithm for many machine learning engineers and practitioners in the data science community.

Let's see how well XGBoost performs on the given race game data set.

## 2. Feature Engineering (ELO)
Adding ELO as essential feature to our data. We will only consider the ELO (estimate of player strength) of the previous day.

In [ ]:
import pandas as pd
import numpy as np

from data.transformation import compute_elo, extract_player_info


# %%
data = pd.read_csv("data/cleaned_data/races_cleaned.csv")